<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:white; background-image:url("https://storage.googleapis.com/kaggle-competitions/kaggle/34377/logos/header.png?t=2022-02-11-21-53-06"); border:0' role="tab" aria-controls="home">
<left><font size = '20'>SpaceShip Titanic Solutions</font></left></h3>

# Predict which passengers are transported to an alternate dimension
In this competition your task is to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly.<br> To help you make these predictions, you're given a set of personal records recovered from the ship's damaged computer system.
<br>
To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly using records recovered from the spaceship’s damaged computer system.
<br>

<a href="https://ibb.co/R91rqc6"><img src="https://i.ibb.co/Y36nxhd/2022-02-24-225001.jpg" alt="2022-02-24-225001" border="0"></a>
<br>
# To Anyone reading This Notebook.
- I hope this Notebook is useful for studying Marchine Learning.<br>I think if you learn hardly you will get great Gift.<br>If you get any Question or give FeedBack, Put comment!! I will answer in detail

<b>- If useful, Don't forget vote!</b>
# Content
- Load Dataset
- Data Visualization and Data Explore
- Data PreProcessing
- Modeling
- R2 Visualization
- Reference

**Features**

`PassengerId` - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.

`HomePlanet` - The planet the passenger departed from, typically their planet of permanent residence.

`CryoSleep` - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.

`Cabin` - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.

`Destination` - The planet the passenger will be debarking to.

`Age` - The age of the passenger.

`VIP` - Whether the passenger has paid for special VIP service during the voyage.

`RoomService, FoodCourt, ShoppingMall, Spa, VRDeck` - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.

`Name` - The first and last names of the passenger.

`Transported` - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:white; background-image:url("https://storage.googleapis.com/kaggle-competitions/kaggle/34377/logos/header.png?t=2022-02-11-21-53-06"); border:0' role="tab" aria-controls="home">
<left><font size = '6'>Load Dataset</font></left></h3>

In [226]:
!pip install pycaret

# Load Dataset
- The Python Pandas packages helps us work with our datasets. We start by acquiring the training and testing datasets into Pandas DataFrames.

In [227]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import re

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%matplotlib inline

# machine learning
from pycaret.classification import *

# Describing Data
<b>- Categorical Feature : HomePlanet, CryoSleep, Cabin, Destination, VIP, Name</b>

<b>- Numerical Feature : RoomService, FoodCount, ShoppingMall, Spa, VRDeck</b>
<br>

* (PassengerId is unique Id for each passenger. It doesn't importance making Model.)

In [220]:
train_df = pd.read_csv('../input/spaceship-titanic/train.csv')
test_df = pd.read_csv('../input/spaceship-titanic/test.csv')

train_df.head()

In [174]:
train_df.describe()

<b>Train Data, Test Data have a lot of NaN. We have to do some prprocessing.</b>

In [118]:
train_df.info()
print()
print('--------'*6)
test_df.info()

# Pivoting Features
- HomePlanet : Since most of Data is Earth, We will replace NaN values with Earth.
- Destionation : Since most of Data is Earth, We will replace NaN values with TRAPPIST-1e.
- VIP : Most of them are False, so we will relpace them with False as well
<br>

- Cabin : Cabin is consists of Deck, Num, Side. It can be divided and analyzed
- CryoSleep : NaN replace False as well.

In [119]:
print(train_df.columns.values)
print('------'*6)
print(train_df['HomePlanet'].value_counts())
print('------'*6)
print(train_df['Destination'].value_counts())
print('------'*6)
print(train_df['VIP'].value_counts())
print('------'*6)
print(train_df['Transported'].value_counts())
print('------'*6)
print(train_df['Cabin'].str[0].value_counts())
print('------'*6)
print(train_df['CryoSleep'].value_counts())

In [120]:
train_df[['HomePlanet', 'Transported']].groupby(['HomePlanet'], as_index = False).mean().sort_values(by = 'Transported', ascending = False)

In [121]:
train_df[['Destination', 'Transported']].groupby(['Destination'], as_index = False).mean().sort_values(by = 'Transported', ascending = False)

In [122]:
train_df[['VIP', 'Transported']].groupby(['VIP'], as_index = False).mean().sort_values(by = 'Transported', ascending = False)

<b>The ratio of the target value may be different for each age distribution, so we use the cut of pandas to figure it out.</b>
<br>
But.. There is no many difference in the ratio. I'll put it on hold for now

In [123]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
print(train_df[['AgeBand', 'Transported']].groupby(['AgeBand'], as_index = False).mean().sort_values(by = 'AgeBand', ascending = True))
train_df = train_df.drop('AgeBand', axis = 1)

<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:white; background-image:url("https://storage.googleapis.com/kaggle-competitions/kaggle/34377/logos/header.png?t=2022-02-11-21-53-06"); border:0' role="tab" aria-controls="home">
<left><font size = '6'>Data Visualization</font></left></h3>

# Data Visualiztion and Analyze Data
- It's can bring you insight that are not numerically invisible.


In [124]:
total = 4602 + 2131 + 1759
ear = 4602
eur = 2131
mars = 1759
labels = 'Earth', 'Europa','Mars' 
sizes = [ear/total*100, eur/total*100, mars/total*100]
explode = (0.1, 0, 0.1)

total_de = 5915 + 1800 + 796
TRAPPIST = 5915
Cancri = 1800
PSOJ318 = 796
labels_1 = 'TRAPPIST-1e', '55 Cancri e', 'PSO J318.5-22'
sizes_1 = [TRAPPIST/total_de*100, Cancri/total_de*100, PSOJ318/total_de*100]
explode_1 = (0, 0.2, 0)
colors = ['#ff9999','#66b3ff','#99ff99' ]
plt.figure(figsize = (20, 10))
plt.subplot(1, 2, 1)
plt.pie(sizes, explode = explode, labels = labels, autopct='%1.1f%%',
        shadow=True, startangle=90,  textprops={'fontsize': 14}, colors=colors)
plt.axis('equal')
plt.title('HomePlanet Ratio')
plt.subplot(1, 2, 2)
plt.pie(sizes_1, explode = explode_1, labels = labels_1,  autopct='%1.1f%%',
        shadow=True, startangle=90,  textprops={'fontsize': 14}, colors=colors)
plt.axis('equal')
plt.title('Destination Ratio')
plt.suptitle('HomePlanet & Destination Ratio', fontsize = 25)
plt.show()

In [125]:
fig = px.scatter_3d(train_df, x = 'Spa', y = 'VRDeck', z = 'RoomService', color = 'RoomService')
fig.show()
#RoomService, Spa, VRDeck

In [126]:
fig = px.scatter_3d(train_df, x = 'Name', y = 'VIP', z = 'Age', color = 'Age')
fig.show()

In [127]:
g = sns.FacetGrid(train_df, col = 'Transported')
g.map(plt.hist, 'Age', bins = 20)
plt.show()

In [128]:
grid = sns.FacetGrid(train_df, col = 'Transported', row = 'HomePlanet', height = 2.2, aspect = 1.6)
grid.map(plt.hist, 'Age', alpha = .5, bins = 20)
grid.add_legend()
plt.show()

In [129]:
grid = sns.FacetGrid(train_df, col = 'Transported', row = 'VIP', height = 2.2, aspect = 1.6)
grid.map(plt.hist, 'Age', alpha = .5, bins = 20)
grid.add_legend()
plt.show()

<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:white; background-image:url("https://storage.googleapis.com/kaggle-competitions/kaggle/34377/logos/header.png?t=2022-02-11-21-53-06"); border:0' role="tab" aria-controls="home">
<left><font size = '6'>Data PreProcessing</font></left></h3>

# Data PreProcessing
- To train model, We need to conver the categorical data into numeric form. We will use, Label Encoder.
- I will divid Cabin to Deck, Num and side
- PassengerId, Cabin, Name is not Importance to train model. therefore drop Features

In [221]:
missing_feature_freq = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Name']
missing_feature_mean = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Age']
sub_id = test_df['PassengerId']

def missing_fill(df):
    for feature in missing_feature_freq:
        most_freq = df[feature].value_counts().index[0]
        df[feature] = df[feature].fillna(most_freq)
    
    for feature in missing_feature_mean:
        df[feature] = df[feature].fillna(df[feature].mean())
    
    return df

def extract_deck(s):
    return s.split('/')[0]

def extract_num(s):
    return s.split('/')[1]

def extract_side(s):
    return s.split('/')[2]

# def extract_surname(s):
#     return s.split()[1]

def drop_features(df):
    drop_feature = ['PassengerId', 'Cabin', 'Name']
    for ft in drop_feature:
        df = df.drop(ft, axis = 1)
    return df
def encoder(df):
    dum_features = ['CryoSleep', 'HomePlanet', 'Destination', 'VIP', 'Deck', 'Side'] # surname
    
    for dum in dum_features:
        le = LabelEncoder()
        le.fit(df[dum])
        df[dum] = le.transform(df[dum])
        
    df['Num'] = df['Num'].astype('int')
    return df

def Make_feature(df):
    df = missing_fill(df)
    df['Total Charge'] = np.log1p(df['RoomService'] + df['FoodCourt'] + df['ShoppingMall'] + df['Spa'] + df['VRDeck'])
    df['Deck'] = df['Cabin'].apply(extract_deck)
    df['Num'] = df['Cabin'].apply(extract_num)
    df['Side'] = df['Cabin'].apply(extract_side)
#     df['surname'] = df['Name'].apply(extract_surname)
    df = drop_features(df)
    df = encoder(df)
    
    return df

train_df = Make_feature(train_df)
test_df = Make_feature(test_df)

In [195]:
plt.figure(figsize = (20, 10))
plt.subplot(1, 2, 1)
plt.hist(np.log1p(train_df['Total Charge']), bins = 100)
plt.title('Train Data Total Charge')
plt.subplot(1, 2, 2)
plt.hist(np.log1p(test_df['Total Charge']), bins = 100)
plt.title('test Data Total Charge')
plt.suptitle('Train & Test Data Total Charge', fontsize = 20)
plt.show()

In [162]:
# Total Charge
grid = sns.FacetGrid(train_df, col = 'VIP', hue = 'Transported')
grid.map(plt.hist, 'Total Charge', alpha = .5, bins = 20)
grid.add_legend()
plt.show()

In [163]:
grid = sns.FacetGrid(train_df, row = 'HomePlanet', col = 'Transported', height = 2.2, aspect = 1.6)
grid.map(sns.barplot, 'VIP',  'Total Charge', alpha = .5, order = None)
grid.add_legend()
plt.show()

<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:white; background-image:url("https://storage.googleapis.com/kaggle-competitions/kaggle/34377/logos/header.png?t=2022-02-11-21-53-06"); border:0' role="tab" aria-controls="home">
<left><font size = '6'>Modeling with Pycaret</font></left></h3>

# Modeling
PyCaret is an open-source, low-code machine learning library in Python that automates machine learning workflows.<br> It is an end-to-end machine learning and model management tool that exponentially speeds up the experiment cycle and makes you more productive.
<br>

Link : https://pycaret.gitbook.io/docs/

In [228]:
model = setup(data = train_df, target = 'Transported', session_id=123, silent = True)

In [232]:
top4_models = compare_models(fold = 10, n_select= 4)

In [234]:
blender = blend_models(top4_models)

In [235]:
# tmp = tune_model(blender, n_iter = 100)

<b>Model Score

<a href="https://ibb.co/9vH7VbS"><img src="https://i.ibb.co/z5Vz7XD/2022-02-24-221944.jpg" alt="2022-02-24-221944" border="0"></a>

In [237]:
# tmp_predict = predict_model(tmp, data = test_df)
# tmp_predict.head()

# submission = pd.DataFrame({
#     'PassengerId' : sub_id,
#     'Transported' : tmp_predict['Label']
# })
# submission.to_csv('submission.csv', index = False)

In [239]:
# catboost_model = create_model('catboost', fold = 10, round = 6)

In [240]:
# cat_tune = tune_model(catboost_model, n_iter = 100)

<a href="https://ibb.co/R91rqc6"><img src="https://i.ibb.co/Y36nxhd/2022-02-24-225001.jpg" alt="2022-02-24-225001" border="0"></a>

In [242]:
# tmp_predict = predict_model(catboost_model, data = test_df)

# submission = pd.DataFrame({
#     'PassengerId' : sub_id,
#     'Transported' : tmp_predict['Label']
# })
# submission.to_csv('submission2.csv', index = False)

<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:white; background-image:url("https://storage.googleapis.com/kaggle-competitions/kaggle/34377/logos/header.png?t=2022-02-11-21-53-06"); border:0' role="tab" aria-controls="home">
<left><font size = '6'>Reference Link</font></left></h3>

# Reference
- https://www.kaggle.com/startupsci/titanic-data-science-solutions/notebook
- https://www.kaggle.com/kalelpark/herbarium2022-eda-with-pytorch
- https://www.kaggle.com/drcapa/spaceship-titanic-starter
- https://www.kaggle.com/c/spaceship-titanic/discussion/309535

<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:white; background-image:url("https://storage.googleapis.com/kaggle-competitions/kaggle/34377/logos/header.png?t=2022-02-11-21-53-06"); border:0' role="tab" aria-controls="home">
<left><font size = '20'>To be Continue</font></left></h3>

In [ ]:
train_df['Cabin'].value_counts()